In [1]:
import numpy as np
import sys
import cv2
import os

In [2]:
path = "./DNN/googlenet/fig/"
filelist = os.listdir(path)
filenames = []
for i in filelist:
    filename = path + i
    filenames.append(filename)
    
model = "./DNN/googlenet/bvlc_googlenet.caffemodel"
config = "./DNN/googlenet/deploy.prototxt"

net = cv2.dnn.readNet(model, config)

if net.empty():
    print("net read failed")
    sys.exit()
    
classnames = []
with open("./DNN/googlenet/classification_classes_ILSVRC2012.txt", "rt") as f:
    classnames = f.read().rstrip("\n").split("\n")
    
idx = 0
while 1:
    img = cv2.imread(filenames[idx])
    
    if img is None:
        break
    # color에 대한 scalefactor, 칼라를 스케일링 했냐
    # mean = color normalization
    blob = cv2.dnn.blobFromImage(img, 1, (224, 224), (104, 117, 123), 
                                 swapRB = False)
    
    net.setInput(blob)
    prob = net.forward()
    
    out = prob.flatten()
    classid = np.argmax(out)
    confidence = out[classid]
    name = classnames[classid]
    
    text = "{},{}%".format(name, round(confidence * 100, 2))
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
               (0, 0, 255), 1, cv2.LINE_AA)
    
    cv2.imshow("img", img)
    
    idx += 1
    
    if idx >= len(filenames):
        idx = 0
        
    if cv2.waitKey(2000) == 27:
        break

cv2.destroyAllWindows()

### face recognition

In [16]:
import numpy as np
import sys
import cv2
import os

In [6]:
model = "./DNN/opencv_face_detector/opencv_face_detector_uint8.pb"
config = "./DNN/opencv_face_detector/opencv_face_detector.pbtxt"

net = cv2.dnn.readNet(model, config)

if net.empty():
    sys.exit("net read failed")
    
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    sys.exit("video read failed")
    
while 1:
    ret, img = cap.read()
    
    if not ret:
        break
    blob = cv2.dnn.blobFromImage(img, 1, (300, 300), (104, 177, 123),
                                swapRB = False)
    net.setInput(blob)
    out = net.forward() # ()가 공란이면 마지막 인덱스에 결과
    
    detect = out[0, 0, :, :]
    h, w = img.shape[:2]
    
    for i in range(detect.shape[0]):
        confidence = detect[i, 2]
        
        if confidence > 0.5:
            x1 = int(detect[i, 3] * w)
            y1 = int(detect[i, 4] * h)
            x2 = int(detect[i, 5] * w)
            y2 = int(detect[i, 6] * h)
            
            cv2.circle(img, (int((x2 + x1)/2), int((y2 + y1)/2)), 
                       int(np.linalg.norm()), (0, 0, 255), 5, cv2.LINE_AA)
#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
            text = "face, {}%".format(round(confidence * 100, 2))
            
            cv2.putText(img, text, (x1, y1 - 3), cv2.FONT_HERSHEY_SIMPLEX,
                       1, (0, 0, 255), 2, cv2.LINE_AA)
    
    cv2.imshow("img", img)
    
    if cv2.waitKey(20) == 27:
        break
            
cap.release()
cv2.destroyAllWindows()

### yolo3

In [9]:
import glob

In [ ]:
model = "./DNN/yolo_v3/yolov3.weights"
cfg = "./DNN/yolo_v3/yolov3.cfg"
cls_ = "./DNN/yolo_v3/coco.names"

imgfiles = glob.glob("./DNN/yolo_v3/fig/*.jpg")

net = cv2.dnn.readNet(model, cfg)

if net.empty():
    sys.exit("net read failed")
    
l_class = []
with open(cls_) as f:
    l_class = f.read().rstrip("\n").rsplit("\n")
    
colors = np.random.uniform(0, 255, size = (80, 3))

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

for i in imgfiles:
    img = cv2.imread(imgfiles[i])

    
    blob = cv2.dnn.blobFromImage(img, 1/255. (320, 320), (0, 0, 0),
                                swapRB = True)
    
    net.setInput(blob)
    outs = net.forward(output_layers)
    
    h, w = img.shape[:2]
    
    class_ids = []
    confidences = []
    boxes = []
    
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > 0.5:
                cx = detection[0] * w
                cy = detection[1] * h
                bw = int(detection[2] * w)
                bh = int(detection[3] * h)
                
                sx = int(cx - bw / 2)
                sy = int(cy - bh / 2)
                
                boxes.append([sx, sy, bw, bh])
                confidences.append(float(confidence))
                class_ids.append(int(class_id))
                
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    
    for i in indices:
        sx, sy, bw, bh = boxes[i]
        label = "{} : {}%".format(l_class[class_ids[i]], 
                                  round(confidences[i] * 100, 2))
        color = colors[class_ids[i]]
        cv2.rectangle(img, (sx, sy, bw, bh), color, 2)
        cv2.putText(img, label, (sx, sy - 3), cv2.FONT_HERSHEY_SIMPLEX,
                   0.7, color, 1, cv2.LINE_AA)
        
    t, _ = net.getPerfProfile() # 주사선 속도 반환
    label = "inference time : {}".format(t / cv2.getTickFrequency() * 1000)
    cv2.putText(img, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1,
               (255, 0, 0), 1, cv2.LINE_AA)
    
    cv2.imshow("img", img)
    
    if cv2.waitKey(3000) == 27:
        break
        
cv2.destroyAllWindows()

In [ ]:
model = './yolo_v3/yolov3.weights'
config = './yolo_v3/yolov3.cfg'
class_label = './yolo_v3/coco.names'

import glob

img_files = glob.glob('./yolo_v3/fig/.jpg')

cap = cv2.VideoCapture('./fig/video2.mp4')
if not cap.isOpened():
    print('video open failed')
    sys.exit()

net = cv2.dnn.readNet(model, config)
if net.empty():
    sys.exit("Net 404")
Class = []
with open('./yolo_v3/coco.names', 'rt') as f:
    Class = f.read().rstrip('\n').split('\n')

Color = np.random.uniform(0, 255, size = (80,3))
# print(Color.shape)

Layers = net.getLayerNames()
# print(Layers)
output = [Layers[i-1] for i in net.getUnconnectedOutLayers()]
# print(output)

while True:
    ret, img = cap.read()
    blob = cv2.dnn.blobFromImage(img, 1/255., (320, 320), (104, 117, 123), swapRB = True)

    net.setInput(blob)
    Outs = net.forward(output)
# print(Outs[0].shape)
    h, w = img.shape[:2]

    Classid =  []
    Confidence = []
    Bx = []

    for out in Outs:
        for det in out:
            score = det[5:]
            classid = np.argmax(score)
            confidence = score[classid]

            if confidence >= 0.5:
                cx = (det[0] * w)
                cy = (det[1] * h)
                bw = int(det[2] * w)
                bh = int(det[3] * h)

                sx = int(cx - bw // 2)
                sy = int(cy - bh // 2)

                Bx.append([sx, sy, bw, bh])
                Confidence.append(float(confidence))
                Classid.append(classid)
inD = cv2.dnn.NMSBoxes(Bx, Confidence, 0.5 ,0.4)
    for i in inD:
        sx, sy, bw, bh = Bx[i]
        label = '{} : {}%'.format(Class[Classid[i]], round(Confidence[i]*100,2))
        color = Color[Classid[i]]
        cv2.rectangle(img, (sx,sy,bw,bh), color, 2)
        cv2.putText(img, label, (sx, sy-10), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 150), 1, cv2.LINEAA)

    t,  = net.getPerfProfile()
    label = 'inference time: {}'.format(round(t/cv2.getTickFrequency() * 1000, 2))
    cv2.putText(img, label, (10, 30), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 155), 1, cv2.LINE_AA)

    cv2.imshow('img', img)

    if cv2.waitKey(2500) == 27:
        break
cap.release()
cv2.destroyAllWindows()

### shear

In [17]:
import sys
import numpy as np
import cv2

In [36]:
src = cv2.imread("./fig/lenna.bmp", 1)

h, w = src.shape[:2]

affine = np.array([[1, 0.5, 0],
                   [0, 1, 0]], np.float32)

dst = cv2.warpAffine(src, affine, (int(w * 1.5), h))

cv2.imshow("src", src)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [44]:
src = cv2.imread("./fig/checkerboard.png")

src_point = np.array([[217, 50], [691, 47], [830, 517], [67, 526]], np.float32)
dst_point = np.array([[0, 0], [w - 1, 0], [w - 1, h - 1], [0, h - 1]], np.float32)

pers_coor = cv2.getPerspectiveTransform(src_point, dst_point)
dst = cv2.warpPerspective(src, pers_coor, (w, h))

cv2.imshow("src", src)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

In [47]:
# 실습
src = cv2.imread("./fig/visitor_card.png")

src_point = np.array([[42, 73], [239, 25], [285, 122], [73, 194]], np.float32)
dst_point = np.array([[0, 0], [w - 1, 0], [w - 1, h - 1], [0, h - 1]], np.float32)

pers_coor = cv2.getPerspectiveTransform(src_point, dst_point)
dst = cv2.warpPerspective(src, pers_coor, (w, h))

cv2.imshow("src", src)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

### 이미지를 흐물흐물하게

In [63]:
src = cv2.imread("./fig/bamboo.jpg")

h, w = src.shape[:2]
map2, map1 = np.indices((h, w), dtype = np.float32)
map1 = map1 + 10 * np.tan(map2 / 32)

dst = cv2.remap(src, map1, map2, cv2.INTER_CUBIC, borderMode = cv2.BORDER_DEFAULT)

cv2.imshow("src", src)
cv2.imshow("dst", dst)

cv2.waitKey()
cv2.destroyAllWindows()

### object tracking

In [1]:
import sys
import numpy as np
import cv2

In [2]:
cap = cv2.VideoCapture("./DNN/PETS2000.avi")

retback, back = cap.read()
if not retback:
    print("background read failed")

back = cv2.cvtColor(back, cv2.COLOR_BGR2GRAY)
back = cv2.GaussianBlur(back, (0, 0), 1.0)
fback = back.astype(np.float32)


while 1:
    ret, frame = cap.read()
    
    if not ret:
        print("i r f")
        sys.exit()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (0, 0), 1.0)
    
    cv2.accumulateWeighted(gray, fback, 0.001)
    back = fback.astype(np.uint8)
    
#     frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray, back)
    _, diff_th = cv2.threshold(diff, 30, 255, cv2.THRESH_BINARY)
    
    cnts, _, stats, _ = cv2.connectedComponentsWithStats(diff_th)
    
    for i in range(1, cnts):
        x, y, w, h, s = stats[i]
        
        if s < 300:
            continue
        cv2.rectangle(frame, (x, y, w, h), (0, 0, 255), 2)
    
    cv2.imshow("frame", frame)
    cv2.imshow("back", back)
    cv2.imshow("diff", diff)
    cv2.imshow("diff_th", diff_th)
    
    if cv2.waitKey(30) == 27:
        break

cap.release()
cv2.destroyAllWindows()